## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering


## OLD STUFF IGNORE

In [2]:
df = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')
df

C:\Users\sathi\AppData\Local\Temp\ipykernel_20632\870708999.py:1: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,18247,149.171.126.4,7662,tcp,FIN,0.119596,4550,68342,31,...,,6,2,2,5,1,1,2,NaN,0
1,59.166.0.3,54771,149.171.126.2,27709,tcp,FIN,0.650574,8928,320,31,...,,3,5,2,4,1,1,4,NaN,0
2,59.166.0.8,13289,149.171.126.9,5190,tcp,FIN,0.007980,2158,2464,31,...,,3,5,1,1,1,1,3,NaN,0
3,149.171.126.18,1043,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,,19,19,19,19,19,19,19,NaN,0
4,149.171.126.18,1043,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,,19,19,19,19,19,19,19,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540042,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,...,,1,2,3,3,1,1,3,NaN,0
2540043,59.166.0.7,20848,149.171.126.4,21,tcp,CON,0.365058,456,346,31,...,2,2,2,2,2,2,2,2,NaN,0
2540044,59.166.0.3,21511,149.171.126.9,21,tcp,CON,6.335154,1802,2088,31,...,2,2,2,4,2,2,2,2,NaN,0
2540045,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,...,,1,1,2,4,2,2,2,NaN,0


In [21]:
%%capture --no-display
df.groupby('proto')['Label'].value_counts()

proto      Label
3pc        1        137
a/n        1        137
aes-sp3-d  1        137
any        1        411
argus      1        137
                   ... 
wsn        1        137
xnet       1        137
xns-idp    1        137
xtp        1        137
zero       1        137
Name: count, Length: 138, dtype: int64

In [19]:
%%capture --no-stdout --no-display
print(df1)

proto      Label
3pc        1        137
a/n        1        137
aes-sp3-d  1        137
any        1        411
argus      1        137
                   ... 
wsn        1        137
xnet       1        137
xns-idp    1        137
xtp        1        137
zero       1        137
Name: count, Length: 138, dtype: int64


In [ ]:
#group by service, and counts with label
df1 = df.groupby('proto')['Label'].value_counts()

 Cleaning dataset

Drop is_sm_ips_ports

In [3]:
#Cleaning df by removing rows with is_sm_ips_ports = 1
df = df[df['is_sm_ips_ports']==0]
#Dropping is_sm_ips_ports column
df.drop(['is_sm_ips_ports'], axis=1, inplace=True)

C:\Users\sathi\AppData\Local\Temp\ipykernel_16936\1444029592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['is_sm_ips_ports'], axis=1, inplace=True)


In [4]:
#Find out unique values in df['attack_cat']
df['attack_cat'].unique()

array([nan, 'Exploits', ' Fuzzers ', 'DoS', 'Generic', ' Shellcode ',
       'Backdoor', ' Reconnaissance ', 'Worms', 'Analysis',
       'Reconnaissance', 'Shellcode', ' Fuzzers', 'Backdoors'],
      dtype=object)

In [5]:
#Remove trailing white spaces from the dataset
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [6]:
df['attack_cat'].unique()

array([nan, 'Exploits', 'Fuzzers', 'DoS', 'Generic', 'Shellcode',
       'Backdoor', 'Reconnaissance', 'Worms', 'Analysis', 'Backdoors'],
      dtype=object)

In [7]:
#Replace 'Backdoor' with 'Backdoors'
df['attack_cat'] = df['attack_cat'].replace('Backdoor', 'Backdoors')

In [8]:
#Fill NaN values with 0 for int and float columns and None for object columns
for col in df.columns:
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        df[col] = df[col].fillna(0)
    elif df[col].dtype == 'object':
        df[col] = df[col].fillna('None')

In [9]:
df['attack_cat'].unique()

array(['None', 'Exploits', 'Fuzzers', 'DoS', 'Generic', 'Shellcode',
       'Backdoors', 'Reconnaissance', 'Worms', 'Analysis'], dtype=object)

In [10]:
#Replace None in sport and dport with 0
df['sport'] = df['sport'].replace('None', '0')
df['dsport'] = df['dsport'].replace('None', '0')

In [11]:
df.is_ftp_login.unique()

array([0., 1., 4., 2.])

In [12]:
#Convert is_ftp_login to int
df['is_ftp_login'] = df['is_ftp_login'].astype(int)

In [13]:
#Convert all is_ftp_login values > 0 to 1
df['is_ftp_login'] = df['is_ftp_login'].apply(lambda x: 1 if x > 0 else x)

In [14]:
df.ct_dst_src_ltm .unique()

array([ 2,  4,  3, 19,  1,  5,  7,  6, 25, 24, 10,  9, 18, 22, 17, 32, 54,
       15, 33, 48, 40, 37, 16, 42, 34, 39, 31, 36, 11, 30, 21, 27, 29, 35,
       43, 59, 23, 47, 20, 14, 38, 56, 45,  8, 51, 26, 41, 28, 49, 12, 46,
       13, 44, 53, 50, 52, 57, 58, 55, 60, 66, 61, 65, 67, 63],
      dtype=int64)

In [15]:
#Convert to csv and store in a new file
df.to_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv', index=False)

In [16]:
df.drop(['attack_cat'],axis=1,inplace=True)

Label Encoding

In [17]:
#label encoding for non numerical features
labelencoder = LabelEncoder()
from sklearn import preprocessing 

# label_encoder object knows  
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

df['proto'] = labelencoder.fit_transform(df['proto'])
df['service'] = labelencoder.fit_transform(df['service'])
df['state'] = labelencoder.fit_transform(df['state'])
df['attack_cat'] = labelencoder.fit_transform(df['attack_cat'])



KeyError: 'attack_cat'

In [ ]:
df2 = pd.read_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv',encoding='cp1252')
df2

C:\Users\sathi\AppData\Local\Temp\ipykernel_20212\128579614.py:1: DtypeWarning: Columns (1,3,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv',encoding='cp1252')


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,0,149.171.126.4,7662,tcp,FIN,0.119596,4550,68342,31,...,NaN,6,2,2,5,1,1,2,None,0
1,59.166.0.3,0,149.171.126.2,27709,tcp,FIN,0.650574,8928,320,31,...,NaN,3,5,2,4,1,1,4,None,0
2,59.166.0.8,0,149.171.126.9,5190,tcp,FIN,0.007980,2158,2464,31,...,NaN,3,5,1,1,1,1,3,None,0
3,149.171.126.18,0,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,NaN,19,19,19,19,19,19,19,None,0
4,149.171.126.18,0,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,NaN,19,19,19,19,19,19,19,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535847,59.166.0.5,0,149.171.126.7,0,tcp,FIN,0.087306,320,1828,31,...,NaN,1,2,3,3,1,1,3,None,0
2535848,59.166.0.7,0,149.171.126.4,0,tcp,CON,0.365058,456,346,31,...,2.0,2,2,2,2,2,2,2,None,0
2535849,59.166.0.3,0,149.171.126.9,0,tcp,CON,6.335154,1802,2088,31,...,2.0,2,2,4,2,2,2,2,None,0
2535850,59.166.0.9,0,149.171.126.0,0,tcp,CON,2.200934,3498,166054,31,...,NaN,1,1,2,4,2,2,2,None,0


In [ ]:

#Replace '' and 'None' with 0 in ct_ftp_cmd
df2['ct_ftp_cmd'] = df2['ct_ftp_cmd'].replace('None', '0')
df2['ct_ftp_cmd'] = df2['ct_ftp_cmd'].replace('', '0')

In [ ]:
df2.ct_ftp_cmd.fillna(0, inplace=True)

In [ ]:
df2['ct_ftp_cmd'] = df2['ct_ftp_cmd'].astype(int)

In [ ]:
df2['ct_ftp_cmd'].unique()

array([0, 1, 4, 2, 6])

In [ ]:
#Convert to csv and store in a new file
df2.to_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv', index=False)

In [ ]:
#convert ct_ftp_cmd to int
df['ct_ftp_cmd'] = df['ct_ftp_cmd'].astype(int)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2535852 entries, 0 to 2540046
Data columns (total 48 columns):
 #   Column            Dtype  
---  ------            -----  
 0   srcip             object 
 1   sport             object 
 2   dstip             object 
 3   dsport            object 
 4   proto             int32  
 5   state             int32  
 6   dur               float64
 7   sbytes            int64  
 8   dbytes            int64  
 9   sttl              int64  
 10  dttl              int64  
 11  sloss             int64  
 12  dloss             int64  
 13  service           int32  
 14  Sload             float64
 15  Dload             float64
 16  Spkts             int64  
 17  Dpkts             int64  
 18  swin              int64  
 19  dwin              int64  
 20  stcpb             int64  
 21  dtcpb             int64  
 22  smeansz           int64  
 23  dmeansz           int64  
 24  trans_depth       int64  
 25  res_bdy_len       int64  
 26  Sjit          

In [ ]:
#Convert to csv and store in a new file
df.to_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv', index=False)

Filling in null values

In [ ]:
#Fill nullvalues
#Filling Nan values with 0
df.fillna(0, inplace=True)
#Filling '' with 0
df.replace('', 0, inplace=True)
df.replace(' ', 0, inplace=True)

C:\Users\sathi\AppData\Local\Temp\ipykernel_2968\201358963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)
C:\Users\sathi\AppData\Local\Temp\ipykernel_2968\201358963.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace('', 0, inplace=True)
C:\Users\sathi\AppData\Local\Temp\ipykernel_2968\201358963.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(' ', 0, inplace=True)


In [ ]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,59.166.0.1,18247,149.171.126.4,7662,114,5,0.119596,4550,68342,31,...,0.0,0,6,2,2,5,1,1,2,0
1,59.166.0.3,54771,149.171.126.2,27709,114,5,0.650574,8928,320,31,...,0.0,0,3,5,2,4,1,1,4,0
2,59.166.0.8,13289,149.171.126.9,5190,114,5,0.007980,2158,2464,31,...,0.0,0,3,5,1,1,1,1,3,0
3,149.171.126.18,1043,175.45.176.3,53,120,6,0.000005,264,0,60,...,0.0,0,19,19,19,19,19,19,19,0
4,149.171.126.18,1043,175.45.176.3,53,120,6,0.000005,264,0,60,...,0.0,0,19,19,19,19,19,19,19,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540042,59.166.0.5,33094,149.171.126.7,43433,114,5,0.087306,320,1828,31,...,0.0,0,1,2,3,3,1,1,3,0
2540043,59.166.0.7,20848,149.171.126.4,21,114,2,0.365058,456,346,31,...,2.0,2,2,2,2,2,2,2,2,0
2540044,59.166.0.3,21511,149.171.126.9,21,114,2,6.335154,1802,2088,31,...,2.0,2,2,2,4,2,2,2,2,0
2540045,59.166.0.9,35433,149.171.126.0,80,114,2,2.200934,3498,166054,31,...,0.0,0,1,1,2,4,2,2,2,0


Correlation with label

In [ ]:
#Correlation with output variable
cor = df.corr()
cor_target = abs(cor["Label"])
cor_target

proto               0.011832
state               0.338312
dur                 0.002055
sbytes              0.010134
dbytes              0.075698
sttl                0.904512
dttl                0.134569
sloss               0.043423
dloss               0.096054
service             0.009492
Sload               0.192121
Dload               0.220163
Spkts               0.121179
Dpkts               0.116195
swin                0.316271
dwin                0.315079
stcpb               0.234160
dtcpb               0.234252
smeansz             0.065710
dmeansz             0.273102
trans_depth         0.029195
res_bdy_len         0.027398
Sjit                0.020903
Djit                0.054562
Stime               0.275647
Ltime               0.275647
Sintpkt             0.012086
Dintpkt             0.010668
tcprtt              0.143044
synack              0.122190
ackdat              0.143386
ct_state_ttl        0.880216
ct_flw_http_mthd    0.026690
is_ftp_login        0.031908
ct_srv_src    

In [ ]:
sorted(cor_target,reverse=True)

[1.0,
 0.9045123713273255,
 0.8802159911646181,
 0.4397582040181459,
 0.41894609595431975,
 0.3966171141155974,
 0.3862785180682943,
 0.38276987066060003,
 0.34300251941699716,
 0.33923406199653117,
 0.3383115509958634,
 0.31627126136549794,
 0.31507938791330836,
 0.2756467745810105,
 0.2756467361934781,
 0.2731022353548177,
 0.23425165967570538,
 0.23415972437915167,
 0.22016301877421282,
 0.19212078068924934,
 0.14338606748651497,
 0.14304379120018665,
 0.1345687229012209,
 0.12218956387082323,
 0.12117868665702373,
 0.11619454701110364,
 0.09605449315010073,
 0.07569769679289255,
 0.06571002635878419,
 0.054562098246944746,
 0.043423355305197676,
 0.03190817000838083,
 0.02919529475181047,
 0.02739821746445534,
 0.02668985508780704,
 0.020902958909795925,
 0.01208573079157584,
 0.011832106432080278,
 0.010667773256950744,
 0.010133696212686666,
 0.009492262382872188,
 0.002055001885478537]

In [ ]:
#Information Gain of features
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

X = df.iloc[:,0:42]  #independent columns
y = df.iloc[:,-1]    #target column i.e Label


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

In [ ]:
#SAmple the dataset
# Randomly sample 1000 rows
dfsample = df.sample(n=1000, random_state=42)

# Extract features and labels
src_encoder = LabelEncoder()
x = src_encoder.fit_transform(x)
dst_encoder = LabelEncoder()
y = dst_encoder.fit_transform(data['Dst IP Addr'])

x = torch.tensor(dfsample[['srcip', 'dstip']].values, dtype=torch.long)
y = torch.tensor(dfsample['Label'].values, dtype=torch.long)

features = dfsample.drop(columns=['srcip', 'dstip','sport','dsport', 'Label'])
scaler = StandardScaler()
features = scaler.fit_transform(features)

#Sampling
# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Construct edge indices
edge_index = torch.tensor([dfsample['srcip'].values, dfsample['dstip'].values], dtype=torch.long)
edge_attr = torch.tensor(features, dtype=torch.ListType) 

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        return x

# Instantiate the model
model = GCN(input_dim=2, hidden_dim=64, output_dim=2)  # input_dim=2 since we have 2 features (source and destination IPs), output_dim=2 for binary edge classification

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train_model(model, optimizer, criterion, x_train, edge_index, edge_attr, y_train, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(x_train, edge_index, edge_attr)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()
        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

# Convert edge labels to one-hot encoding
y_train_onehot = F.one_hot(y_train)

# Train the model
train_model(model, optimizer, criterion, x_train, edge_index, edge_attr, y_train, epochs=10)

TypeError: tensor(): argument 'dtype' must be torch.dtype, not pybind11_type

In [ ]:
# Testing
def test_model(model, x_test, edge_index, edge_attr, y_test):
    model.eval()
    with torch.no_grad():
        out = model(x_test, edge_index, edge_attr)
        pred = out.argmax(dim=1)
        f1 = f1_score(y_test, pred, average='macro')
        print(f'F1 Score: {f1}')

# Test the model
test_model(model, x_test, edge_index, edge_attr, y_test)

## Import Dataset

In [2]:
df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')
#Set attack_cat as -1 if Label ==0 else attack_cat
df['attack_cat'] = df.apply(lambda x: -1 if x['Label'] == 0 else x['attack_cat'], axis=1)


C:\Users\sathi\AppData\Local\Temp\ipykernel_18892\868858865.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')


In [ ]:
#Split df into 5 minutes timeframe based on Stime

start_Stime = 1421927443    
df_timeframe_1_5mins = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 300))]


In [13]:
#Count the number of times each attack_cat repeats
df['attack_cat'].value_counts()

-1    2214569
 5     215481
 3      44525
 4      24246
 2      16353
 7      13987
 0       2677
 1       2329
 8       1511
 9        174
Name: attack_cat, dtype: int64

-1 Non attack ||
5 Generic ||
3 Exploits ||
4 Fuzzers ||
2 DoS ||
7 Reconnaissance ||
0 Analysis ||
1 Backdoors ||
8 Shellcode ||
9 Worms

Alphabetical ordering is followed during label encoding

In [3]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,0,149.171.126.4,7662,114,5,0.119596,4550,68342,31,...,0,6,2,2,5,1,1,2,6,0
1,59.166.0.3,0,149.171.126.2,27709,114,5,0.650574,8928,320,31,...,0,3,5,2,4,1,1,4,6,0
2,59.166.0.8,0,149.171.126.9,5190,114,5,0.007980,2158,2464,31,...,0,3,5,1,1,1,1,3,6,0
3,149.171.126.18,0,175.45.176.3,53,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
4,149.171.126.18,0,175.45.176.3,53,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535847,59.166.0.5,0,149.171.126.7,0,114,5,0.087306,320,1828,31,...,0,1,2,3,3,1,1,3,6,0
2535848,59.166.0.7,0,149.171.126.4,0,114,2,0.365058,456,346,31,...,2,2,2,2,2,2,2,2,6,0
2535849,59.166.0.3,0,149.171.126.9,0,114,2,6.335154,1802,2088,31,...,2,2,2,4,2,2,2,2,6,0
2535850,59.166.0.9,0,149.171.126.0,0,114,2,2.200934,3498,166054,31,...,0,1,1,2,4,2,2,2,6,0


In [14]:
df.to_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv', index=False)

In [ ]:
#Compare df and df1, and replace values of sport and dsport in df with values in df1


In [3]:
#Count the number of records with Label = 1 and Label = 0 for each unique value of sttl and ct_state_ttl
counts = df.groupby(['sttl', 'ct_state_ttl', 'Label']).size().unstack(fill_value=0)

# Displaying the counts
print("Counts of Label = 0 and Label = 1 for each unique combination of sttl and ct_state_ttl:")
print(counts)

Counts of Label = 0 and Label = 1 for each unique combination of sttl and ct_state_ttl:
Label                    0       1
sttl ct_state_ttl                 
0    0                5862       0
     2                  23     375
1    0                4520       0
29   0                2736       0
30   0                 225       0
31   0             1940700       0
32   0                1602       0
60   0              214255       0
62   0                  19       1
     1                3847   22231
     2                   0     180
     3                3394     291
63   0                 524       0
     1                   1       1
64   0                3625       0
252  0                  84       0
254  0                2888    1078
     1               19623   35684
     2                7200  260037
     3                 454     112
     4                  12      28
     5                   1       1
     6                2935    1228
255  0                  37      29
  

In [26]:
len(counts)

25

In [4]:
countssttl = df.groupby(['sttl', 'Label']).size().unstack(fill_value=0)

# Displaying the counts
print("Counts of Label = 0 and Label = 1 for each unique sttl:")
print(countssttl)

Counts of Label = 0 and Label = 1 for each unique sttl:
Label        0       1
sttl                  
0         5885     375
1         4520       0
29        2736       0
30         225       0
31     1940700       0
32        1602       0
60      214255       0
62        7260   22703
63         525       1
64        3625       0
252         84       0
254      33113  298168
255         39      36


In [11]:
df['attack_cat'].unique()

array([6, 3, 4, 2, 5, 8, 1, 7, 9, 0], dtype=int64)

In [14]:
countsattcatsttl = df.groupby(['sttl', 'attack_cat']).size().unstack(fill_value=0)

# Displaying the counts
print("Counts of attack_cat for each unique sttl:")
print(countsattcatsttl)

Counts of attack_cat for each unique sttl:
attack_cat       -1     0     1      2      3      4       5      7     8    9
sttl                                                                          
0              5885     3    21    108    183     15      21     24     0    0
1              4520     0     0      0      0      0       0      0     0    0
29             2736     0     0      0      0      0       0      0     0    0
30              225     0     0      0      0      0       0      0     0    0
31          1940700     0     0      0      0      0       0      0     0    0
32             1602     0     0      0      0      0       0      0     0    0
60           214255     0     0      0      0      0       0      0     0    0
62             7260   609    47   1828  18725     55    1395     40     0    4
63              525     0     0      0      1      0       0      0     0    0
64             3625     0     0      0      0      0       0      0     0    0
252      

In [22]:
df1 = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')

C:\Users\sathi\AppData\Local\Temp\ipykernel_8472\3150562644.py:1: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')


In [23]:
#Replace 'Backdoor' with 'Backdoors'
#Remove trailing white spaces from the dataset
df1 = df1.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df1['attack_cat'] = df1['attack_cat'].replace('Backdoor', 'Backdoors')

In [25]:
countsattcatsttl1 = df1.groupby(['sttl', 'attack_cat']).size().unstack(fill_value=0)

# Displaying the counts
print("Counts of attack_cat for each unique sttl:")
print(countsattcatsttl1)

Counts of attack_cat for each unique sttl:
attack_cat  Analysis  Backdoors    DoS  Exploits  Fuzzers  Generic  \
sttl                                                                 
0                  3         21    108       183       15       21   
62               609         47   1828     18725       55     1395   
63                 0          0      0         1        0        0   
254             2065       2261  14401     25612    24161   214065   
255                0          0     16         4       15        0   

attack_cat  Reconnaissance  Shellcode  Worms  
sttl                                          
0                       24          0      0  
62                      40          0      4  
63                       0          0      0  
254                  13922       1511    170  
255                      1          0      0  


In [6]:
countsctstatettl = df.groupby(['ct_state_ttl', 'Label']).size().unstack(fill_value=0)

# Displaying the counts
print("Counts of Label = 0 and Label = 1 for each unique 'ct_state_ttl':")
print(countsctstatettl)

Counts of Label = 0 and Label = 1 for each unique 'ct_state_ttl':
Label               0       1
ct_state_ttl                 
0             2177077    1108
1               23473   57923
2                7223  260592
3                3848     403
4                  12      28
5                   1       1
6                2935    1228


## Graph EDA

In [3]:
#FInd minimum and maximum value of Stime and Ltime
minstime = df['Stime'].min()
minltime = df['Ltime'].min()
maxstime = df['Stime'].max()
maxltime = df['Ltime'].max()

print("Stime : ", minstime, " to ", maxstime, " difference = ", maxstime - minstime )
print("Ltime : ", minltime, " to ", maxltime , " difference = ", maxltime-minltime)

Stime :  1421927377  to  1424262068  difference =  2334691
Ltime :  1421927414  to  1424262069  difference =  2334655


In [5]:
# Convert Unix time to DD/MM/YYYY format
from datetime import datetime
minstimets = datetime.fromtimestamp(minstime).strftime('%d/%m/%Y')
minstimets

'22/01/2015'

In [6]:
maxstimets = datetime.fromtimestamp(maxstime).strftime('%d/%m/%Y')
maxstimets

'18/02/2015'

In [8]:
top_ten_stime = df['Stime'].value_counts().head(10)
print(top_ten_stime)


1421927443    426
1424219889    383
1421945101    226
1424251722    225
1424237259    213
1421945117    210
1421927444    209
1424230476    202
1424251558    201
1421945200    200
Name: Stime, dtype: int64


In [4]:
#Set attack_cat = -1 when label = 0
df['attack_cat'] = df.apply(lambda x: -1 if x['Label'] == 0 else x['attack_cat'], axis=1)

In [5]:
df.attack_cat.value_counts()

-1    2214569
 5     215481
 3      44525
 4      24246
 2      16353
 7      13987
 0       2677
 1       2329
 8       1511
 9        174
Name: attack_cat, dtype: int64

In [6]:
df.sport.value_counts()

0        2445148
0          12944
47439       2398
1043        2054
47439        339
          ...   
65254          1
57073          1
4077           1
33654          1
15104          1
Name: sport, Length: 47691, dtype: int64

In [15]:
df.dsport.value_counts()

0        916027
53       617298
80       126521
5190      64255
6881      61833
          ...  
26781         1
22261         1
25630         1
24029         1
30870         1
Name: dsport, Length: 69442, dtype: int64

In [2]:
df1 = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')

C:\Users\sathi\AppData\Local\Temp\ipykernel_11172\3150562644.py:1: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')


In [17]:
df1.attack_cat.value_counts()

Generic             215481
Exploits             44525
 Fuzzers             19195
DoS                  16353
 Reconnaissance      12228
 Fuzzers              5051
Analysis              2677
Backdoor              1795
Reconnaissance        1759
 Shellcode            1288
Backdoors              534
Shellcode              223
Worms                  174
Name: attack_cat, dtype: int64

In [8]:
df1.sport.value_counts()

1043     217979
47439    200532
0         49884
0          4160
138        3480
          ...  
13623         1
34404         1
51737         1
46837         1
706           1
Name: sport, Length: 107594, dtype: int64

In [16]:
df1.dsport.value_counts()

53       622508
53       176460
80       128153
80        98545
5190      65271
          ...  
8996          1
53066         1
10514         1
30579         1
11764         1
Name: dsport, Length: 128286, dtype: int64

In [18]:
len(df1[df1['sport'] == 'None'])

0

In [19]:
len(df1[df1['sport'] == 'NaN'])

0

In [3]:
#Add all unique 'srcip' + 'sport' combinations to a set
srcip_sport_set = set()
for index, row in df1.iterrows():
    srcip_sport_set.add((row['srcip'], row['sport']))
    

In [4]:
#Split df1 into df1_label_true and df1_label_false
df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]

In [26]:
df1_label_true.Label.value_counts()

1    321283
Name: Label, dtype: int64

In [27]:
df1_label_false.Label.value_counts()

0    2218764
Name: Label, dtype: int64

In [5]:
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')


In [31]:
df1_label_false_grouped

,srcip,sport,dstip,dsport,count
0,10.40.170.2,0,10.40.170.2,0,1194
1,10.40.170.2,0,10.40.170.2,0,802
2,10.40.170.2,0,10.40.170.2,0,20
3,10.40.170.2,0,10.40.170.2,0,78
4,10.40.182.1,0,10.40.182.3,0,998
...,...,...,...,...,...
1899350,59.166.0.9,9952,149.171.126.7,53,1
1899351,59.166.0.9,9979,149.171.126.9,143,1
1899352,59.166.0.9,9989,149.171.126.6,80,1
1899353,59.166.0.9,9993,149.171.126.5,38742,1


In [6]:
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')


In [33]:
df1_label_true_grouped

,srcip,sport,dstip,dsport,count
0,175.45.176.0,0,149.171.126.10,0,299
1,175.45.176.0,0,149.171.126.11,0,24
2,175.45.176.0,0,149.171.126.11,0,240
3,175.45.176.0,0,149.171.126.12,0,320
4,175.45.176.0,0,149.171.126.13,0,5188
...,...,...,...,...,...
65730,175.45.176.3,9750,149.171.126.11,80,1
65731,175.45.176.3,9761,149.171.126.17,21,1
65732,175.45.176.3,9812,149.171.126.15,80,1
65733,175.45.176.3,9909,149.171.126.13,80,1


In [7]:
import networkx as nx
# Create an empty graph
attackG = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    attackG.add_node(src_node)
    attackG.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    attackG.add_edge(src_node, dst_node, weight=edge_weight)


In [8]:
import networkx as nx
# Create an empty graph
nonAttackG = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonAttackG.add_node(src_node)
    nonAttackG.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]
    nonAttackG.add_edge(src_node, dst_node, weight=edge_weight)

In [9]:
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

In [10]:
import networkx as nx
# Create an empty graph
overallG = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG.add_node(src_node)
    overallG.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG.add_edge(src_node, dst_node, weight=edge_weight)

In [11]:
import networkx as nx
# Calculate centrality measures
degree_centrality = nx.degree_centrality(attackG)
closeness_centrality = nx.closeness_centrality(attackG)
betweenness_centrality = nx.betweenness_centrality(attackG)
eigenvector_centrality = nx.eigenvector_centrality(attackG)
number_of_communities = nx.number_connected_components(attackG)

# Calculate average centrality scores
average_degree = sum(degree_centrality.values()) / len(degree_centrality)
average_closeness = sum(closeness_centrality.values()) / len(closeness_centrality)
average_betweenness = sum(betweenness_centrality.values()) / len(betweenness_centrality)
average_eigenvector = sum(eigenvector_centrality.values()) / len(eigenvector_centrality)

# Print the results
print("Degree Centrality:", degree_centrality)
print("Average Closeness Centrality:", average_closeness)
print("Average Betweenness Centrality:", average_betweenness)
print("Average Eigenvector Centrality:", average_eigenvector)
print("Number of Communities:", number_of_communities)


KeyboardInterrupt: 

In [ ]:
clustering_coefficient = nx.clustering(attackG)
# Average Shortest Path Length
average_shortest_path_length = nx.average_shortest_path_length(attackG)
# Global Efficiency
global_efficiency = nx.global_efficiency(attackG)
# Local Efficiency
local_efficiency = nx.local_efficiency(attackG)
# Transitivity
transitivity = nx.transitivity(attackG)
# Diameter
diameter = nx.diameter(attackG)
# Average Clustering
average_clustering = nx.average_clustering(attackG)
# Density
density = nx.density(attackG)

print("Density:", density)
print("Clustering Coefficient:", clustering_coefficient)
print("Average Shortest Path Length:", average_shortest_path_length)
print("Global Efficiency:", global_efficiency)
print("Local Efficiency:", local_efficiency)
print("Transitivity:", transitivity)
print("Average Clustering:", average_clustering)

In [13]:
density = nx.density(attackG)

In [14]:
density

1.686936433800373e-05

In [42]:
# Save the graph to Gephi file format
nx.write_gexf(overallG, "D:\Project Phase II\Graphs\completegraph.gexf")


In [44]:
nx.write_gexf(attackG, "D:\Project Phase II\Graphs\\attackgraph.gexf")

In [45]:
nx.write_gexf(nonAttackG, "D:\Project Phase II\Graphs\\nonattackgraph.gexf")

## EDA

In [4]:
nodes = set()
for i in range(len(df)):
    nodes.add(str(df['srcip'][i])+':'+str(df['sport'][i]))
    nodes.add(str(df['srcip'][i])+':'+str(df['sport'][i]))

In [5]:
len(nodes)

68192

## Baseline ML Models

In [7]:
#Use logistic regression to predict the label
#Split the dataset into features and labels
X = df['sttl']
y = df['Label']

#Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Logistic regression to classify only using one variable
LR = LogisticRegression()
LR.fit(X_train.values.reshape(-1,1), y_train)

y_pred = LR.predict(X_test.values.reshape(-1,1))



In [9]:
#Confusion matrix
confusion_matrix(y_test, y_pred)


array([[436361,   6577],
       [  4599,  59634]], dtype=int64)

In [11]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    442938
           1       0.90      0.93      0.91     64233

    accuracy                           0.98    507171
   macro avg       0.95      0.96      0.95    507171
weighted avg       0.98      0.98      0.98    507171



In [8]:
#F1 score for each label
f1_score(y_test, y_pred, average=None)

array([0.98735601, 0.91432339])

In [12]:
#PCA for dimensionality reduction using only two variables sttl and ct_state_ttl
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X = df[['sttl', 'ct_state_ttl']]
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca_x_train = pca.fit_transform(X_train)
pca_x_test = pca.fit_transform(X_test)
classifier = LogisticRegression()
classifier.fit(pca_x_train, y_train)
y_pred = classifier.predict(pca_x_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99    442938
           1       0.90      0.93      0.91     64233

    accuracy                           0.98    507171
   macro avg       0.95      0.96      0.95    507171
weighted avg       0.98      0.98      0.98    507171



In [13]:
#Linear equation formed by PCA
print(pca.components_)

[[0.99996518 0.00834444]]


## Find Prediction Values of Baseline ML Models Alone

In [5]:
#Find all string values in sport
df['sport'] = df['sport'].astype(str)
#Replace '-' with 0
df['sport'] = df['sport'].replace('-', '80')
#COnvert hexadecimal values to decimal in sport
df['sport'] = df['sport'].apply(lambda x: int(x, 16))
#Convert sport to int
df['sport'] = df['sport'].astype(int)

len(df['sport'].unique())

43005

In [6]:
df['dsport'] = df['dsport'].astype(str)
df['dsport'] = df['dsport'].replace('-', '80')
df['dsport'] = df['dsport'].apply(lambda x: int(x, 16))
df['dsport'] = df['dsport'].astype(int)
len(df['dsport'].unique())


64380

In [7]:
dfSample0 = df[df['Label'] == 0].sample(n=250000, random_state=42)
dfSample1 = df[df['Label'] == 1].sample(n=250000, random_state=42)

dfSample = pd.concat([dfSample0, dfSample1])
dfSample = dfSample.sample(frac=1, random_state=42).reset_index(drop=True)

#Split the dataset into features and labels
XSample = dfSample.drop('Label', axis=1)
ySample = dfSample['Label']

#Split the dataset into training and testing sets
XSample_train, XSample_test, ySample_train, ySample_test = train_test_split(XSample, ySample, test_size=0.2, random_state=42)


In [8]:
dfSample0 = df[df['Label'] == 0].sample(n=500000, random_state=42)
dfSample1 = df[df['Label'] == 1]

In [9]:
dfSample0 = dfSample0.drop(['srcip', 'dstip', 'attack_cat','Stime','Ltime'], axis=1)
dfSample0 = dfSample0.drop_duplicates()
dfSample1 = dfSample1.drop(['srcip', 'dstip', 'attack_cat','Stime','Ltime'], axis=1)
dfSample1 = dfSample1.drop_duplicates()

print("Number of unique records in the DataFrame for Label = 0:", len(dfSample0))
print("Number of unique records in the DataFrame for Label = 1:", len(dfSample1))

Number of unique records in the DataFrame for Label = 0: 454722
Number of unique records in the DataFrame for Label = 1: 79055


In [10]:
dfSample0 = dfSample0.sample(n=79055,random_state=40)

Train0,Test0 = train_test_split(dfSample0, test_size=0.2, random_state=40)
Train1,Test1 = train_test_split(dfSample1, test_size=0.2, random_state=40)

Train = pd.concat([Train0,Train1])
Test = pd.concat([Test0,Test1])

Train = Train.sample(frac=1, random_state=40).reset_index(drop=True)
Test = Test.sample(frac=1, random_state=40).reset_index(drop=True)

X_train = Train.drop('Label', axis=1)
y_train = Train['Label']
X_test = Test.drop('Label', axis=1)
y_test = Test['Label']


Logistic Regression

In [41]:
#Logistic regression to classify using all variables
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.74      0.82     15811
           1       0.78      0.94      0.86     15811

    accuracy                           0.84     31622
   macro avg       0.86      0.84      0.84     31622
weighted avg       0.86      0.84      0.84     31622



In [42]:
#Decision tree classifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     15811
           1       0.99      0.98      0.99     15811

    accuracy                           0.99     31622
   macro avg       0.99      0.99      0.99     31622
weighted avg       0.99      0.99      0.99     31622



Lazy Predict

In [43]:
#Use lazypredict 
from lazypredict.Supervised import LazyClassifier
#Remove time consuming lazypredict classifier models from training


In [44]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm

#Skip SVM
classifiers = [
 ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LinearDiscriminantAnalysis',  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('LogisticRegression', sklearn.linear_model._logistic.LogisticRegression),
 ('Perceptron', sklearn.linear_model._perceptron.Perceptron),
 ('QuadraticDiscriminantAnalysis',  sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis),
 ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
 ('StackingClassifier', sklearn.ensemble._stacking.StackingClassifier),
 ('XGBClassifier', xgboost.sklearn.XGBClassifier),
 ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers=classifiers,predictions=True)


In [22]:
from lazypredict import Supervised
Supervised.CLASSIFIERS

[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('ExtraTreeClassifier', sklearn.tree._classes.ExtraTreeClassifier),
 ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',
  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LabelPropagation',
  sklearn.semi_supervised._label_propagation.LabelPropagation),
 ('LabelSpreading', sklearn.semi_supervised._label_propagation.LabelSpreading),
 ('LinearDiscriminantAnalysis',
  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('Linear

In [45]:
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


  0%|          | 0/15 [00:00<?, ?it/s]

  7%|▋         | 1/15 [01:31<21:18, 91.29s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.992663335652394, 'Balanced Accuracy': 0.992663335652394, 'ROC AUC': 0.992663335652394, 'F1 Score': 0.992662999630621, 'Time taken': 91.28628396987915}


 13%|█▎        | 2/15 [02:52<18:28, 85.24s/it]

...

 93%|█████████▎| 14/15 [12:28<00:35, 35.06s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.9929163240781734, 'Balanced Accuracy': 0.9929163240781734, 'ROC AUC': 0.9929163240781733, 'F1 Score': 0.9929159935505343, 'Time taken': 13.57119870185852}
[LightGBM] [Info] Number of positive: 63244, number of negative: 63244
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.211749 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6513
[LightGBM] [Info] Number of data points in the train set: 126488, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 15/15 [12:44<00:00, 50.97s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.9929163240781734, 'Balanced Accuracy': 0.9929163240781734, 'ROC AUC': 0.9929163240781735, 'F1 Score': 0.9929159811948223, 'Time taken': 16.169971466064453}


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.99,0.99,0.99,0.99,13.57
LGBMClassifier,0.99,0.99,0.99,0.99,16.17
RandomForestClassifier,0.99,0.99,0.99,0.99,114.65
AdaBoostClassifier,0.99,0.99,0.99,0.99,91.29
KNeighborsClassifier,0.99,0.99,0.99,0.99,407.99
LogisticRegression,0.99,0.99,0.99,0.99,8.47
BaggingClassifier,0.99,0.99,0.99,0.99,81.01
LinearDiscriminantAnalysis,0.99,0.99,0.99,0.99,4.40
BernoulliNB,0.99,0.99,0.99,0.99,2.56


In [13]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.99,0.99,0.99,0.99,32.44
RandomForestClassifier,0.99,0.99,0.99,0.99,195.56
LGBMClassifier,0.99,0.99,0.99,0.99,35.15
AdaBoostClassifier,0.99,0.99,0.99,0.99,347.93
BaggingClassifier,0.99,0.99,0.99,0.99,158.00
SGDClassifier,0.99,0.99,0.99,0.99,8.79
LogisticRegression,0.99,0.99,0.99,0.99,26.86
KNeighborsClassifier,0.99,0.99,0.99,0.99,4007.74
Perceptron,0.99,0.99,0.99,0.99,10.41


In [22]:
clf.predictions

True

In [25]:
predictions

,AdaBoostClassifier,BaggingClassifier,BernoulliNB,DecisionTreeClassifier,DummyClassifier,GaussianNB,KNeighborsClassifier,LinearDiscriminantAnalysis,LogisticRegression,Perceptron,QuadraticDiscriminantAnalysis,RandomForestClassifier,XGBClassifier,LGBMClassifier
0,1,1,1,1,0,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99996,1,1,1,1,0,1,1,1,1,1,1,1,1,1
99997,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99998,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
import joblib
# Save trained models
joblib.dump(predictions, 'lazyclassifier_model_predictions.pkl')

['lazyclassifier_model_predictions.pkl']

In [47]:
from sklearn.metrics import roc_auc_score
for model in predictions.columns:
    print(model)
    print(classification_report(y_test, predictions[model]))
    auc_scores = roc_auc_score(y_test, predictions[model], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier


              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15811
           1       0.99      1.00      0.99     15811

    accuracy                           0.99     31622
   macro avg       0.99      0.99      0.99     31622
weighted avg       0.99      0.99      0.99     31622

AUC Scores for each class: 0.992663335652394
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15811
           1       0.99      1.00      0.99     15811

    accuracy                           0.99     31622
   macro avg       0.99      0.99      0.99     31622
weighted avg       0.99      0.99      0.99     31622

AUC Scores for each class: 0.991524887736386
----------------------------------------------------------------------

In [48]:
#Try without sttl and ct_state_ttl
X_train = X_train.drop(['sttl', 'ct_state_ttl'], axis=1)
X_test = X_test.drop(['sttl', 'ct_state_ttl'], axis=1)

#There's a marginal duplication of 27 records. So assume there is no duplicate records

In [58]:
#Try decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.98      0.99      0.99     15811
           1       0.99      0.98      0.99     15811

    accuracy                           0.99     31622
   macro avg       0.99      0.99      0.99     31622
weighted avg       0.99      0.99      0.99     31622



In [60]:
importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X_train.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 12 Dload (0.610672)
2. feature 7 dttl (0.222591)
3. feature 15 swin (0.052201)
4. feature 5 sbytes (0.036693)
5. feature 3 state (0.022931)
6. feature 9 dloss (0.019652)
7. feature 11 Sload (0.004743)
8. feature 6 dbytes (0.003896)
9. feature 17 stcpb (0.002123)
10. feature 28 synack (0.001867)
11. feature 34 ct_srv_dst (0.001769)
12. feature 18 dtcpb (0.001706)
13. feature 29 ackdat (0.001604)
14. feature 33 ct_srv_src (0.001582)
15. feature 19 smeansz (0.001572)
16. feature 23 Sjit (0.001498)
17. feature 27 tcprtt (0.001451)
18. feature 4 dur (0.001237)
19. feature 39 ct_dst_src_ltm (0.001236)
20. feature 25 Sintpkt (0.001158)
21. feature 36 ct_src_ ltm (0.001126)
22. feature 1 dsport (0.001111)
23. feature 26 Dintpkt (0.000818)
24. feature 37 ct_src_dport_ltm (0.000717)
25. feature 35 ct_dst_ltm (0.000684)
26. feature 2 proto (0.000530)
27. feature 24 Djit (0.000519)
28. feature 20 dmeansz (0.000486)
29. feature 0 sport (0.000428)
30. feature 13 Spkts (0.

In [49]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm

#Skip SVM
classifiers = [
 ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LinearDiscriminantAnalysis',  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('LogisticRegression', sklearn.linear_model._logistic.LogisticRegression),
 ('Perceptron', sklearn.linear_model._perceptron.Perceptron),
 ('QuadraticDiscriminantAnalysis',  sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis),
 ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
 ('StackingClassifier', sklearn.ensemble._stacking.StackingClassifier),
 ('XGBClassifier', xgboost.sklearn.XGBClassifier),
 ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers=classifiers,predictions=True)


In [56]:
models1,predictions1 = clf.fit(X_train, X_test, y_train, y_test)
models1

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


  7%|▋         | 1/15 [01:44<24:21, 104.42s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.9909240402251597, 'Balanced Accuracy': 0.9909240402251597, 'ROC AUC': 0.9909240402251597, 'F1 Score': 0.9909236661801477, 'Time taken': 104.4176733493805}


 13%|█▎        | 2/15 [02:45<17:07, 79.01s/it] 

{'Model': 'BaggingClassifier', 'Accuracy': 0.9917146290557207, 'Balanced Accuracy': 0.9917146290557206, 'ROC AUC': 0.9917146290557207, 'F1 Score': 0.9917144007265325, 'Time taken': 61.21940326690674}


...

 93%|█████████▎| 14/15 [12:24<00:36, 36.54s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.9927898298652836, 'Balanced Accuracy': 0.9927898298652837, 'ROC AUC': 0.9927898298652836, 'F1 Score': 0.9927894871760407, 'Time taken': 13.116864442825317}
[LightGBM] [Info] Number of positive: 63244, number of negative: 63244
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.249554 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6493
[LightGBM] [Info] Number of data points in the train set: 126488, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 15/15 [12:40<00:00, 50.67s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.9928214534185061, 'Balanced Accuracy': 0.992821453418506, 'ROC AUC': 0.9928214534185061, 'F1 Score': 0.9928211027767646, 'Time taken': 15.11256742477417}


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.99,0.99,0.99,0.99,15.11
XGBClassifier,0.99,0.99,0.99,0.99,13.12
RandomForestClassifier,0.99,0.99,0.99,0.99,125.45
BaggingClassifier,0.99,0.99,0.99,0.99,61.22
AdaBoostClassifier,0.99,0.99,0.99,0.99,104.42
KNeighborsClassifier,0.99,0.99,0.99,0.99,409.24
DecisionTreeClassifier,0.99,0.99,0.99,0.99,10.12
LogisticRegression,0.98,0.98,0.98,0.98,8.08
LinearDiscriminantAnalysis,0.98,0.98,0.98,0.98,2.89


In [57]:
from sklearn.metrics import roc_auc_score
for model in predictions1.columns:
    print(model)
    print(classification_report(y_test, predictions1[model]))
    auc_scores = roc_auc_score(y_test, predictions1[model], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     15811
           1       0.98      1.00      0.99     15811

    accuracy                           0.99     31622
   macro avg       0.99      0.99      0.99     31622
weighted avg       0.99      0.99      0.99     31622

AUC Scores for each class: 0.9909240402251597
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15811
           1       0.99      1.00      0.99     15811

    accuracy                           0.99     31622
   macro avg       0.99      0.99      0.99     31622
weighted avg       0.99      0.99      0.99     31622

AUC Scores for each class: 0.9917146290557207
-------------------------------------------------

## Finding Performance of Models in Attack Category Classification

In [12]:
#Find information gain and gini index for each feature in the dataset in classifying the label
#Information gain
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

#IG for each attribute
ig = mutual_info_classif(X_train, y_train, random_state=42)
print(ig)
#Print column name with ig
ig = pd.Series(ig)
ig.index = X_train.columns



[0.00467594 0.28453542 0.01533218 0.15907297 0.38336585 0.58759297
 0.55313286 0.6514427  0.61479295 0.19213226 0.21914381 0.04620834
 0.38123949 0.4259998  0.23815449 0.38012804 0.0054774  0.00667274
 0.00126116 0.00272367 0.39241371 0.44096202 0.01675245 0.06628789
 0.28844532 0.24373788 0.31653359 0.44219682 0.4447345  0.44188447
 0.44460516 0.63879798 0.01560297 0.00144817 0.00072851 0.08059814
 0.11891383 0.17153435 0.13297856 0.03306991 0.03414698 0.04889669]


In [14]:
ig = pd.Series(ig)
ig.index = X_train.columns
ig.sort_values(ascending=True, inplace=True)
ig

ct_ftp_cmd          0.000729
stcpb               0.001261
is_ftp_login        0.001448
dtcpb               0.002724
sport               0.004676
swin                0.005477
dwin                0.006673
proto               0.015332
ct_flw_http_mthd    0.015603
trans_depth         0.016752
ct_src_dport_ltm    0.033070
ct_dst_sport_ltm    0.034147
service             0.046208
ct_dst_src_ltm      0.048897
res_bdy_len         0.066288
ct_srv_src          0.080598
ct_srv_dst          0.118914
ct_src_ ltm         0.132979
state               0.159073
ct_dst_ltm          0.171534
sloss               0.192132
dloss               0.219144
Spkts               0.238154
Djit                0.243738
dsport              0.284535
Sjit                0.288445
Sintpkt             0.316534
Dpkts               0.380128
Sload               0.381239
dur                 0.383366
smeansz             0.392414
Dload               0.426000
dmeansz             0.440962
synack              0.441884
Dintpkt       

In [15]:
#Gini index
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
gini = dt.feature_importances_
gini = pd.Series(gini)
gini.index = X_train.columns
gini.sort_values(ascending=True, inplace=True)
gini

trans_depth         0.000000
dwin                0.000000
is_ftp_login        0.000000
dttl                0.000000
swin                0.000000
ct_ftp_cmd          0.000039
service             0.000061
dloss               0.000094
Dpkts               0.000094
res_bdy_len         0.000148
Spkts               0.000154
ct_flw_http_mthd    0.000162
proto               0.000210
ct_dst_sport_ltm    0.000253
sloss               0.000297
sport               0.000326
dmeansz             0.000328
ct_src_dport_ltm    0.000358
state               0.000483
ct_dst_ltm          0.000634
Sjit                0.000863
Dintpkt             0.000964
ct_dst_src_ltm      0.001053
Dload               0.001110
Djit                0.001149
smeansz             0.001164
ct_srv_dst          0.001333
ct_src_ ltm         0.001350
Sintpkt             0.001377
tcprtt              0.001378
dur                 0.001434
dtcpb               0.001519
ct_srv_src          0.001524
synack              0.001559
sbytes        

In [16]:
#Find out less important features using other metrics
#Feature importance
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X_train.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 7 sttl (0.239618)
2. feature 31 ct_state_ttl (0.145984)
3. feature 13 Dload (0.123297)
4. feature 8 dttl (0.092329)
5. feature 30 ackdat (0.070177)
6. feature 28 tcprtt (0.056903)
7. feature 21 dmeansz (0.050559)
8. feature 29 synack (0.044978)
9. feature 6 dbytes (0.030799)
10. feature 27 Dintpkt (0.029286)
11. feature 24 Sjit (0.013810)
12. feature 3 state (0.012666)
13. feature 4 dur (0.011738)
14. feature 26 Sintpkt (0.009851)
15. feature 5 sbytes (0.009589)
16. feature 15 Dpkts (0.008150)
17. feature 12 Sload (0.008098)
18. feature 37 ct_dst_ltm (0.006068)
19. feature 10 dloss (0.005066)
20. feature 1 dsport (0.004351)
21. feature 25 Djit (0.004003)
22. feature 20 smeansz (0.003856)
23. feature 14 Spkts (0.002592)
24. feature 41 ct_dst_src_ltm (0.002144)
25. feature 36 ct_srv_dst (0.002121)
26. feature 35 ct_srv_src (0.002058)
27. feature 18 stcpb (0.001586)
28. feature 19 dtcpb (0.001446)
29. feature 39 ct_src_dport_ltm (0.001353)
30. feature 38 ct_src

In [19]:
#Mutual Importance

from sklearn.feature_selection import mutual_info_classif

# Calculate mutual information between features and target variable
mi_scores = mutual_info_classif(X_train, y_train)

# Sort features based on mutual information scores
sorted_features = sorted(zip(X_train.columns, mi_scores), key=lambda x: x[1], reverse=True)

# Print feature names and their mutual information scores
for feature, score in sorted_features:
    print(feature, score)

sttl 0.6519307153233845
ct_state_ttl 0.6390593910486492
dttl 0.6149109035440272
sbytes 0.587247442240477
dbytes 0.5524949642117569
tcprtt 0.44455188180960215
ackdat 0.44436722897014924
Dintpkt 0.44306488146966827
dmeansz 0.4414907882363108
synack 0.441002791565152
Dload 0.42590077456632924
smeansz 0.39244389804582913
dur 0.38360421341189976
Sload 0.3812381337542132
Dpkts 0.37926591556465183
Sintpkt 0.31678366701069494
Sjit 0.2880705450728873
dsport 0.2835658105005858
Djit 0.24339257648715495
Spkts 0.2377133739359738
dloss 0.2188002123816457
sloss 0.1896227178751122
ct_dst_ltm 0.16974317384632043
state 0.1601539334834492
ct_src_ ltm 0.1308162912466777
ct_srv_dst 0.11796191941382195
ct_srv_src 0.08258624334071873
res_bdy_len 0.06581436253435591
ct_dst_src_ltm 0.048519102444590345
service 0.04703875325667761
ct_dst_sport_ltm 0.032456743604212246
ct_src_dport_ltm 0.031693136589936666
ct_flw_http_mthd 0.016299095520632134
proto 0.014640910886305925
trans_depth 0.013385361845124288
swin 0.00

## LSTM 

1. LSTM FOR AVG STTL

Sample the dataset (1% sample taken)

In [5]:
# Prepare data (sample data)
# Replace this part with your actual data loading and preprocessing

#Sample 1% of the dataset
dfsample = df.sample(frac=0.01, random_state=42)

nodes = set()
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)

In [6]:
len(nodes)

5325

In [7]:
len(dfsample)

25359

Model Training


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Modify the call method of the LSTMClassifier
class LSTMClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(LSTMClassifier, self).__init__()
        self.lstm = LSTM(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.lstm(x)
        output = self.dense(x)
        return output

# Prepare an adjacency matrix of the nodes
adjacency_matrix = np.zeros((len(nodes), len(nodes)), dtype=float)
count_matrix = np.zeros((len(nodes), len(nodes)), dtype=int)
label_matrix = np.zeros((len(nodes), len(nodes)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    src_index = list(nodes).index(src)
    dst_index = list(nodes).index(dst)
    if dfsample['Label'].iloc[i] == 1:
        if label_matrix[src_index,dst_index] == 0:
            adjacency_matrix[src_index, dst_index] = dfsample['sttl'].iloc[i]
            label_matrix[src_index, dst_index] = 1  
            count_matrix[src_index,dst_index]+= 1
        else:
            adjacency_matrix[src_index, dst_index] = (adjacency_matrix[src_index, dst_index]*count_matrix[src_index,dst_index]+dfsample['sttl'].iloc[i])/(count_matrix[src_index,dst_index]+1)
            count_matrix[src_index,dst_index]+= 1
    else:
        if count_matrix[src_index,dst_index] == 0:
            adjacency_matrix[src_index, dst_index] = dfsample['sttl'].iloc[i]
            count_matrix[src_index,dst_index]+= 1
        else:
            adjacency_matrix[src_index, dst_index] = (adjacency_matrix[src_index, dst_index]*count_matrix[src_index,dst_index]+dfsample['sttl'].iloc[i])/(count_matrix[src_index,dst_index]+1)
            count_matrix[src_index,dst_index]+= 1

# Flatten tuple values in the adjacency matrix
adjacency_matrix_flat = adjacency_matrix.reshape(-1, 1)
label_flat = label_matrix.reshape(-1, 1)

# Hyperparameters
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification

# Convert adjacency_matrix_flat to a TensorFlow tensor
adjacency_tensor = tf.convert_to_tensor(adjacency_matrix_flat, dtype=tf.float32)

# Initialize model, optimizer, and loss function
model = LSTMClassifier(input_shape, hidden_units, output_units)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Training
history = model.fit(adjacency_tensor, label_flat, epochs=2, batch_size=32, validation_split=0.2)

# Evaluation
#loss, accuracy = model.evaluate(adjacency_tensor, label_flat) 
#print(f'Loss: {loss}, Accuracy: {accuracy}')





Epoch 1/2


708891/708891 [==============================] - 5248s 7ms/step - loss: 2.7351e-04 - accuracy: 1.0000 - val_loss: 2.3519e-05 - val_accuracy: 1.0000
Epoch 2/2
 14804/886114 [..............................] - ETA: 1:04:36 - loss: 1.2331e-06 - accuracy: 1.0000

KeyboardInterrupt: 

Preparting Testing data 

In [21]:
#Take a sample from the dataset to evalute the lstm model
dftestsample = df.sample(frac=0.001, random_state=32)

nodestest = set()
for i in range(len(dftestsample)):
    src = str(dftestsample['srcip'].iloc[i])+':'+str(dftestsample['sport'].iloc[i])
    dst = str(dftestsample['dstip'].iloc[i])+':'+str(dftestsample['dsport'].iloc[i])
    nodestest.add(src)
    nodestest.add(dst)

# Prepare an adjacency matrix of the nodes
adjacency_matrix_test = np.zeros((len(nodestest), len(nodestest)), dtype=float)
label_matrix_test = np.zeros((len(nodestest), len(nodestest)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dftestsample)):
    src = str(dftestsample['srcip'].iloc[i])+':'+str(dftestsample['sport'].iloc[i])
    dst = str(dftestsample['dstip'].iloc[i])+':'+str(dftestsample['dsport'].iloc[i])
    src_index = list(nodestest).index(src)
    dst_index = list(nodestest).index(dst)

    if dftestsample['Label'].iloc[i] == 1:
        if label_matrix_test[src_index,dst_index] == 0:
            adjacency_matrix_test[src_index, dst_index] = dftestsample['sttl'].iloc[i]
            label_matrix_test[src_index, dst_index] = 1  

    else:
        adjacency_matrix_test[src_index, dst_index] = dftestsample['sttl'].iloc[i]


# Flatten tuple values in the adjacency matrix
adjacency_matrix_test_flat = adjacency_matrix_test.reshape(-1, 1)
label_test_flat = label_matrix_test.reshape(-1, 1)

In [17]:
len(nodestest)

732

In [18]:
len(dftestsample)

2536

Model Evaluation

In [22]:
predictions = model.predict(adjacency_matrix_test_flat)
predictions = (predictions > 0.5).astype(int)



14792/14792 [==============================] - 72s 5ms/step


In [23]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictions)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    473252
         1.0       0.76      0.99      0.86        92

    accuracy                           1.00    473344
   macro avg       0.88      0.99      0.93    473344
weighted avg       1.00      1.00      1.00    473344



Refer Phase2ParallelRunning.ipynb for other LSTM and GRU

# Ignore (Was supposed to be for GCN)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import f1_score

# Define GCN layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support) + self.bias
        return output

# Define GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(input_dim, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, adj):
        h = self.relu(self.gc1(x, adj))
        out = self.gc2(h, adj)
        return out

# Function to calculate F1 score
def calculate_f1_score(labels, predictions):
    f1_score_0 = f1_score(labels, predictions, pos_label=0)
    f1_score_1 = f1_score(labels, predictions, pos_label=1)
    return f1_score_0, f1_score_1

# Prepare data (sample data)
# Replace this part with your actual data loading and preprocessing

# Assuming adjacency matrix is already prepared
adjacency_matrix = torch.tensor([[0, 1, 1],
                                 [1, 0, 1],
                                 [1, 1, 0]], dtype=torch.float32)

# Assuming feature matrix is already prepared
feature_matrix = torch.tensor([[0.1, 0.2],
                               [0.3, 0.4],
                               [0.5, 0.6]], dtype=torch.float32)

# Assuming labels are already prepared
labels = torch.tensor([0, 1, 0], dtype=torch.long)

# Hyperparameters
input_dim = feature_matrix.shape[1]
hidden_dim = 16
output_dim = 2
lr = 0.01
epochs = 100

# Initialize model, optimizer, and loss function
model = GCN(input_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(feature_matrix, adjacency_matrix)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(feature_matrix, adjacency_matrix).argmax(dim=1)
    f1_score_0, f1_score_1 = calculate_f1_score(labels.numpy(), predictions.numpy())
    print(f'F1 Score for Label=0: {f1_score_0}')
    print(f'F1 Score for Label=1: {f1_score_1}')